In [ ]:
# langgrpah specific imports 
from langgraph.graph import StateGraph, START, END
from langchain.messages import SystemMessage, HumanMessage, AIMessage
import operator
from langgraph.types import Send
import getpass
import os

from typing import Annotated, TypedDict

In [ ]:


if "GOOGLE_API_KEY" not in os.environ:
    print("Please set the GOOGLE_API_KEY environment variable.")


In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    temperature=1.0,  # Gemini 3.0+ defaults to 1.0
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:

class MessagesState(TypedDict):
    questions: list[str]
    answers: Annotated[list[str], operator.add]


In [37]:
def generate_answers(state: MessagesState) -> MessagesState:
    # This is where you would call your LLM to get an answer to the question.
    # For simplicity, we'll just return a hardcoded answer.
    return {'questions': state['questions']}
    
def send_logic(state: MessagesState) -> MessagesState:

    return [Send("answer_questions",{"question": s}) for s in state['questions']]

def answer_questions(state: MessagesState) -> MessagesState:
    # This is where you would call your LLM to get an answer to the question.
    # For simplicity, we'll just return a hardcoded answer.
    messages = [SystemMessage(content="You are a helpful assistant that answers questions about the world."), HumanMessage(content=state['question'])]
    # answer = f"The answer to '{state['question']}' is ..."
    response = model.invoke(messages)
    return {'answers': [response.text]}


In [38]:
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("generate_answers", generate_answers)
graph_builder.add_node("answer_questions", answer_questions)
graph_builder.add_edge(START,"generate_answers")
graph_builder.add_conditional_edges("generate_answers", send_logic, ["answer_questions"])
graph_builder.add_edge("answer_questions", END)
graph = graph_builder.compile()

In [39]:
graph.invoke({"questions": ["What is the capital of France?", "What is the largest mammal?"]})

{'questions': ['What is the capital of France?',
  'What is the largest mammal?'],
 'answers': ['The capital of France is **Paris**.',
  'The largest mammal is the **blue whale** (*Balaenoptera musculus*).\n\nIn fact, the blue whale is the largest animal known to have ever existed—even larger than the biggest dinosaurs. Here are a few quick facts about its massive size:\n\n*   **Weight:** They can weigh up to **200 tons** (approximately 400,000 pounds).\n*   **Length:** They can grow to nearly **100 feet** (30 meters) long.\n*   **Heart Size:** Their heart is about the size of a bumper car.\n*   **Tongue:** Their tongue alone can weigh as much as an entire elephant.\n\nIf you are looking for the largest **land** mammal, that would be the **African bush elephant**.']}